In [ ]:
# ===============================================================================
# 2. ENGENHARIA DE FEATURES
# ===============================================================================

def feature_engineering(df):
    """Cria novas features relevantes para agricultura"""
    
    print("\n🔧 ENGENHARIA DE FEATURES")
    print("="*40)
    
    df_fe = df.copy()
    
    # 1. Features de Nutrientes
    print("📊 Criando features de nutrientes...")
    df_fe['NPK_Total'] = df_fe['Nitrogen'] + df_fe['POTASH'] + df_fe['PHOSPHATE']
    df_fe['N_P_Ratio'] = df_fe['Nitrogen'] / (df_fe['PHOSPHATE'] + 1)  # +1 para evitar divisão por 0
    df_fe['N_K_Ratio'] = df_fe['Nitrogen'] / (df_fe['POTASH'] + 1)
    df_fe['P_K_Ratio'] = df_fe['PHOSPHATE'] / (df_fe['POTASH'] + 1)
    
    # 2. Features Hídricas
    print("💧 Criando features hídricas...")
    df_fe['Water_Efficiency'] = df_fe['RICE_PRODUCTION'] / (df_fe['ANNUAL'] + 1)
    df_fe['Rain_Intensity'] = df_fe['avg_rain'] / (df_fe['ANNUAL'] / 365)  # chuva diária média
    
    # 3. Índice de Fertilidade
    print("🌱 Criando índice de fertilidade...")
    # Normalizar nutrientes para criar índice
    scaler_temp = StandardScaler()
    nutrients_normalized = scaler_temp.fit_transform(df_fe[['Nitrogen', 'POTASH', 'PHOSPHATE']])
    df_fe['Fertility_Index'] = np.mean(nutrients_normalized, axis=1)
    
    # 4. Tipo de Solo Dominante
    print("🏞️ Identificando solo dominante...")
    soil_cols = [col for col in df_fe.columns if col not in ['ANNUAL', 'avg_rain', 'Nitrogen', 
                                                            'POTASH', 'PHOSPHATE', 'RICE_PRODUCTION']]
    
    # Encontrar solo com maior proporção
    df_fe['Dominant_Soil'] = df_fe[soil_cols].idxmax(axis=1)
    df_fe['Soil_Diversity'] = (df_fe[soil_cols] > 0).sum(axis=1)  # Número de tipos de solo presentes
    
    # 5. Interações importantes
    print("🔗 Criando interações...")
    df_fe['Nitrogen_x_Rain'] = df_fe['Nitrogen'] * df_fe['avg_rain']
    df_fe['NPK_x_Water'] = df_fe['NPK_Total'] * df_fe['avg_rain']
    
    print(f"✅ Features criadas! Shape anterior: {df.shape}, nova: {df_fe.shape}")
    
    return df_fe
